# Artificial Generation of Coherent Melodic Fragments Using Markov Chains
#### By Collin Presser

## Introduction:

As its name suggests, artificial intelligence (AI) largely bases its methods of problem solving in logic and reasoning. From decision trees mirroring the logic of game theory to deep learning and neural networks simulating the processes of the human brain, AI can be used to generate logically sound, understandable, and thus, coherent conclusions or solutions.

Markov chains are one of AI's basic tools for making informed decisions, using probabilities to represent the relationships between states. Their purpose is to approximate an optimal solution for a problem in a feasible amount of time when searching for the most optimal solution is intractable. While they do not possess the same logic or reasoning as the higher concept strategies mentioned before, they can still be used to simulate patterns which are neither arbitrary nor obvious.

Problem solving has many applications in the field of music. From score analysis to multi-part realizations, there are plenty of opportunities where one could use AI in a problem solving capacity. In fact, many music theorists and musicologists have already begun to use AI in these regards (see these articles by <a href='https://www.jstor.org/stable/3680079'>Road</a>, <a href='https://www.jstor.org/stable/10.2307/26350040'>Serra</a>, and <a href='https://www.jstor.org/stable/10.1086/673321'>Wilf</a>). In this project, I will be focusing on AI's application in procedurally generated music.

This tool can be combined with a variety of musical concepts in the creation of procedurally generated music (see Osipenko's <a href='https://towardsdatascience.com/markov-chain-for-music-generation-932ea8a88305'>example</a> using chord progressions). Using tools from the Numpy and Music21 libraries, this project intends to show how Markov chains can be used to create melodic fragments. Using the functions created in this notebook, an experiment can then be designed and prepared to test the coherency of these melodic fragments. In this document, the term "coherent" will be used to describe melodic fragments developed with nuanced patterns that are neither obvious nor arbitrary.

## Resources:

As in all programs, one must start with a basis on which to build. For the design of my Markov chain system, I will be using the Numpy addon. My musical data will be stored in set theory notation/MIDI for ease of use in the program. To visually and aurally represent the music created, I will be using tools from the Music21 API. The last import provides tools for displaying HTML.

In [1]:
import numpy as np
from music21 import *
from ipynb.fs.full.ScoreViewer import showScore

from IPython.core.display import display, HTML

This line formats HTML tables in the document better, and also increases the notebook width:

In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))

These imports are used for the user interface:

In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display as wDisplay
from IPython.display import IFrame

For reproducibility of results in this project, I set my random seed here using the Numpy addon:

In [4]:
seed = 12345

np.random.seed(seed)

## Designing the System:

There are many ways to represent a Markov chain system. The most visually understandable method would be as a network made up of nodes and weighted edges as seen in the image below (courtesy of Alexander Osipenko):

##### <center>Markov chain network for generating chord progressions</center>
![](https://miro.medium.com/max/602/1*YJ_hnlTVaRZVhUq6G_VUhA.png)

The nodes represent states, and each of the edges represents actions which lead to the next action. The weights of the edges represent the probability of that action taking place. These probabilities all must sum to 1 because this system will not work otherwise.

While it certainly is possible to program this system in this manner (and there are many tools in Python that make this easier), a matrix is actually a better way to format chains, as it is easier to code, and resources such as Numpy make manipulating matrices very easy in Python. Here is an example matrix, produced by my program in a previous edition:

<TABLE>
    <TR>
        <TH></TH>
        <TH>C     (0)</TH>
        <TH>C#/Db (1)</TH>
        <TH>D     (2)</TH>
        <TH>D#/Eb (3)</TH>
        <TH>E     (4)</TH>
        <TH>F     (5)</TH>
        <TH>F#/Gb (6)</TH>
        <TH>G     (7)</TH>
        <TH>G#/Ab (8)</TH>
        <TH>A     (9)</TH>
        <TH>A#/Bb (T)</TH>
        <TH>B     (E)</TH>
    </TR>
    <TR>
        <TH>C     (0)</TH>
        <TD>0.12638537934071162</TD>
        <TD>0.01611991068547207</TD>
        <TD>0.14706433682814704</TD>
        <TD>0.13600231376104766</TD>
        <TD>0.09515274105896222</TD>
        <TD>0.05144513737459198</TD>
        <TD>0.15002278650142523</TD>
        <TD>0.00861789121890996</TD>
        <TD>0.13923263142684558</TD>
        <TD>0.04847014389137562</TD>
        <TD>0.05001021664670807</TD>
        <TD>0.03147651126580285</TD>
    </TR>
    <TR>
        <TH>C#/Db (1)</TH>
        <TD>0.08748269034364582</TD>
        <TD>0.12030568947763388</TD>
        <TD>0.11553274661947963</TD>
        <TD>0.05669741419483598</TD>
        <TD>0.09811060123350003</TD>
        <TD>0.06779417959422704</TD>
        <TD>0.03415830430148494</TD>
        <TD>0.12143614611279181</TD>
        <TD>0.0466009664817626</TD>
        <TD>0.08548290540681094</TD>
        <TD>0.11156356105743734</TD>
        <TD>0.054834795176389915</TD>
    </TR>
    <TR>
        <TH>D     (2)</TH>
        <TD>0.07129980893719642</TD>
        <TD>0.03474713381520484</TD>
        <TD>0.07363334777080587</TD>
        <TD>0.11003652575680543</TD>
        <TD>0.07401313012266098</TD>
        <TD>0.16689426203636168</TD>
        <TD>0.022863592350863854</TD>
        <TD>0.12089814454474673</TD>
        <TD>0.13956027087072326</TD>
        <TD>0.0067377048611426005</TD>
        <TD>0.11526644853496844</TD>
        <TD>0.06404963039851984</TD>
    </TR>
    <TR>
        <TH>D#/Eb (3)</TH>
        <TD>0.11767964633459271</TD>
        <TD>0.1386669521238618</TD>
        <TD>0.01876181598155476</TD>
        <TD>0.12031440950267513</TD>
        <TD>0.11430347239265579</TD>
        <TD>0.08509804237830701</TD>
        <TD>0.1267835287889327</TD>
        <TD>0.049861545093572004</TD>
        <TD>0.06761405224562335</TD>
        <TD>0.0996687793616666</TD>
        <TD>0.05563285552179081</TD>
        <TD>0.005614900274767274</TD>
    </TR>
    <TR>
        <TH>E     (4)</TH>
        <TD>0.08754933150852322</TD>
        <TD>0.057652497577700135</TD>
        <TD>0.05251249523759175</TD>
        <TD>0.08159755170017624</TD>
        <TD>0.10830828020287839</TD>
        <TD>0.08810948096556659</TD>
        <TD>0.045402298591841465</TD>
        <TD>0.08431250750662288</TD>
        <TD>0.10823450869221135</TD>
        <TD>0.11081049278733103</TD>
        <TD>0.06176868753062256</TD>
        <TD>0.11374186769893442</TD>
    </TR>
    <TR>
        <TH>F     (5)</TH>
        <TD>0.12604827924219716</TD>
        <TD>0.12983539016945572</TD>
        <TD>0.1124189338339215</TD>
        <TD>0.05101403396863954</TD>
        <TD>0.07011337164652862</TD>
        <TD>0.06770631150939906</TD>
        <TD>0.10731236652841968</TD>
        <TD>0.03544308533224622</TD>
        <TD>0.09893756091141612</TD>
        <TD>0.09836273847052643</TD>
        <TD>0.09102974076899957</TD>
        <TD>0.011778187618250493</TD>
    </TR>
    <TR>
        <TH>F#/Gb (6)</TH>
        <TD>0.06951775758802912</TD>
        <TD>0.15259850989013707</TD>
        <TD>0.00443043328032432</TD>
        <TD>0.07184089754434744</TD>
        <TD>0.021550576271148765</TD>
        <TD>0.15905077504374626</TD>
        <TD>0.12683450775977387</TD>
        <TD>0.18061310245476248</TD>
        <TD>0.05521613483475063</TD>
        <TD>0.13591752609296467</TD>
        <TD>0.020019159350698524</TD>
        <TD>0.002410619889316678</TD>
    </TR>
    <TR>
        <TH>G     (7)</TH>
        <TD>0.19195218739434786</TD>
        <TD>0.056733129558566124</TD>
        <TD>0.17761831968368066</TD>
        <TD>0.11068677744925384</TD>
        <TD>0.15833232170019912</TD>
        <TD>0.08434801199323626</TD>
        <TD>0.08503829733545848</TD>
        <TD>0.009768222876827753</TD>
        <TD>0.02544550685463457</TD>
        <TD>0.02823500185886633</TD>
        <TD>0.03680502439034594</TD>
        <TD>0.03503719890458316</TD>
    </TR>
    <TR>
        <TH>G#/Ab (8)</TH>
        <TD>0.06306794578391835</TD>
        <TD>0.1347645622871345</TD>
        <TD>0.0047571328867394</TD>
        <TD>0.050226639642768224</TD>
        <TD>0.17372794909543016</TD>
        <TD>0.07412346176256306</TD>
        <TD>0.016504809654056526</TD>
        <TD>0.11466684886563056</TD>
        <TD>0.05502996584236384</TD>
        <TD>0.09093394462297959</TD>
        <TD>0.1501450124475683</TD>
        <TD>0.07205172710884765</TD>
    </TR>
    <TR>
        <TH>A     (9)</TH>
        <TD>0.12056128552556084</TD>
        <TD>0.04215383051935601</TD>
        <TD>0.1004997544235946</TD>
        <TD>0.018018987483826045</TD>
        <TD>0.08998269048452155</TD>
        <TD>0.0714624744254769</TD>
        <TD>0.1270381363440811</TD>
        <TD>0.08197209742651322</TD>
        <TD>0.14079231865317582</TD>
        <TD>0.1224988965385218</TD>
        <TD>0.029798146092542794</TD>
        <TD>0.05522138208282918</TD>
    </TR>
    <TR>
        <TH>A#/Bb (T)</TH>
        <TD>0.1156311457236615</TD>
        <TD>0.07861588664399718</TD>
        <TD>0.09976251260259641</TD>
        <TD>0.11886671639791009</TD>
        <TD>0.011160700483520436</TD>
        <TD>0.1030294593007058</TD>
        <TD>0.08677104112738898</TD>
        <TD>0.11726705329014567</TD>
        <TD>0.09806614673046313</TD>
        <TD>0.01239513451877046</TD>
        <TD>0.06451584190105283</TD>
        <TD>0.09391836127978744</TD>
    </TR>
    <TR>
        <TH>B     (E)</TH>
        <TD>0.0018056118982943418</TD>
        <TD>0.14025336402951746</TD>
        <TD>0.007809652143921823</TD>
        <TD>0.03852933477121989</TD>
        <TD>0.05448970659418339</TD>
        <TD>0.1353879051421186</TD>
        <TD>0.022941541639484142</TD>
        <TD>0.07783943344515588</TD>
        <TD>0.1920863375067232</TD>
        <TD>0.08008612990013338</TD>
        <TD>0.14351403225434467</TD>
        <TD>0.1052569506749033</TD>
    </TR>
</TABLE>

In this table, the row represents the original state, while the column represents the next action to be taken. The values in each row are the probabilities of moving to the values of the columns from the value of the row. Each row's probabilities must sum to one, much like all the weights of the edges comming from a node in the network version had to sum to 1.

## Functions for Data Manipulation:

After deciding to use a matrix, the first step is to set up the helper functions for generating or representing chains in this format.

For building matrices based on random values (for testing and initial experimentation), it makes sense to have a function to normalize each row so that they each sum to 1. Even when using data input from a file, having a function like this will help guarantee that the data being used complies with this requirement of the system.

In [5]:
#normalizes the rows of a matrix for Markov chains
def normalize(data):
    sums = data.sum(axis=1)
    for i in range(len(data)):
        data[i, :] *= 1.0/(sums[i])

Next, a function for generating random data is needed. Fortunately, Numpy comes with a function for generating random data in a matrix. Here is an example of its use:

In [6]:
#generate random matrix
matrixExample = np.random.rand(5,5)
print("Randomized Matrix:")
print(matrixExample)

Randomized Matrix:
[[0.92961609 0.31637555 0.18391881 0.20456028 0.56772503]
 [0.5955447  0.96451452 0.6531771  0.74890664 0.65356987]
 [0.74771481 0.96130674 0.0083883  0.10644438 0.29870371]
 [0.65641118 0.80981255 0.87217591 0.9646476  0.72368535]
 [0.64247533 0.71745362 0.46759901 0.32558468 0.43964461]]


The normalization of the rows of this matrix looks likes this:

In [7]:
#normalize matrix
normalize(matrixExample)
print("Normalized Matrix:")
print(matrixExample)

Normalized Matrix:
[[0.42213145 0.14366368 0.0835161  0.09288923 0.25779953]
 [0.16471018 0.2667564  0.18064961 0.20712559 0.18075824]
 [0.35227062 0.45290012 0.00395198 0.0501491  0.14072818]
 [0.16301335 0.2011091  0.21659643 0.23956088 0.17972024]
 [0.24779618 0.27671454 0.18034816 0.12557469 0.16956644]]


Next, to save and load data, file input and output functions should be made. Numpy also has functions which complete these tasks with the use of text files:

In [8]:
#save matrix
np.savetxt("exampleMatrix.txt", matrixExample)

#clear for testing
matrixExample = np.array([[-1]])
print("Redefined matrix:")
print(matrixExample)

#load matrix
matrixExample = np.loadtxt("exampleMatrix.txt")
print("Reloaded matrix:")
print(matrixExample)

Redefined matrix:
[[-1]]
Reloaded matrix:
[[0.42213145 0.14366368 0.0835161  0.09288923 0.25779953]
 [0.16471018 0.2667564  0.18064961 0.20712559 0.18075824]
 [0.35227062 0.45290012 0.00395198 0.0501491  0.14072818]
 [0.16301335 0.2011091  0.21659643 0.23956088 0.17972024]
 [0.24779618 0.27671454 0.18034816 0.12557469 0.16956644]]


Finally, the last general function needed is one that chooses the next state based on the current state. This can easily be done using Numpy's random choice function as seen in this method:

In [9]:
#chooses a value based on the probablities given
def choose(row):
    return np.random.choice(len(row), p = row)

In [10]:
#get row 2 (0-4 indexing)
exampleRow = matrixExample[2]
print("Row 2: " + str(exampleRow))

#choose a value in row 2
exampleChoice = choose(exampleRow)
print("Choice: " + str(exampleChoice))

Row 2: [0.35227062 0.45290012 0.00395198 0.0501491  0.14072818]
Choice: 1


## Functions for Music Representation:

Now that the basic functions have been set up, more functions need to be made to handle the musical elements.

The first function to build will get all the MIDI values of the given pitch class set within a given range:

In [11]:
#gets all of the MIDI numbers that are part of the given pitch class set and are within the range (high is exclusive)
def pcsToMIDI(pcs, low = 0, high = 128):
    result = []
    start = 0
    while(start < high):
        for pc in pcs:
            num = start + pc
            if(num >= low and num < high):
                result.append(num)
            
        start += 12
    
    result.sort()
    return result

In [12]:
#create pcs {5, 9, T}
examplePCS = {5, 9, 10}

#get all MIDI values of {5, 9, T} within B2 (47) and F7 (101)
exampleMIDI = pcsToMIDI(examplePCS, low = 47, high = 101)
print(exampleMIDI)

[53, 57, 58, 65, 69, 70, 77, 81, 82, 89, 93, 94]


Next, these MIDI values can be used with Music21 to get the note names and octaves for clearer representations:

In [13]:
#converts MIDI pitch to scientific pitch notation
def midiToOctaveNotation(midi):
    return note.Note(midi).nameWithOctave + " (" + str(midi) + ")"

In [14]:
#MIDI 70 -> Bb/A#4
print(midiToOctaveNotation(70))

B-4 (70)


Next, here's a function which will print a Markov chain matrix with nice labels:

In [15]:
#prints an HTML table version of the matrix with labels
def printChain(midi, matrix):
    labels = [midiToOctaveNotation(m) for m in midi]
    html = "<TABLE>"
    html += "<TR><TH></TH>"
    for l in labels:
        html += "<TH>" + l + "</TH>"
    
    html += "</TR>"
    for i in range(len(matrix)):
        html += "<TR><TH>" + labels[i] + "</TH>"
        for p in matrix[i]:
            html += "<TD>" + str(p) + "</TD>"
        
        html += "</TR>"
        
    html += "</TABLE>"
    display(HTML(html))

In [16]:
#make a matrix for the MIDI values we got before
midiExampleMatrix = np.random.rand(12, 12)
normalize(midiExampleMatrix)
print(midiExampleMatrix)

#print nice version
printChain(exampleMIDI, midiExampleMatrix)

[[0.16417593 0.11179552 0.13061582 0.02822897 0.00443455 0.13219276
  0.14926289 0.00407563 0.08121921 0.08691868 0.0984985  0.00858154]
 [0.14576113 0.11859473 0.13326446 0.08145893 0.13193554 0.01562802
  0.03565499 0.04213118 0.07622882 0.07480677 0.11554034 0.0289951 ]
 [0.08199023 0.02587868 0.11860992 0.1431949  0.09403055 0.02316978
  0.07553789 0.05821546 0.13091925 0.14056089 0.05921884 0.0486736 ]
 [0.08120194 0.02683207 0.01797798 0.09823133 0.11423344 0.0819366
  0.13903761 0.0905823  0.12692173 0.07077596 0.05023265 0.1020364 ]
 [0.07757943 0.0347367  0.03771359 0.12205095 0.07623134 0.14087784
  0.14553913 0.08209062 0.03887101 0.11097606 0.05656686 0.07676647]
 [0.03201435 0.04995772 0.09196333 0.04421644 0.10861992 0.11047114
  0.1204428  0.13421872 0.01516439 0.12868265 0.04748355 0.11676498]
 [0.16100608 0.00765774 0.03510095 0.05279642 0.11240476 0.01545756
  0.02562632 0.09247427 0.12358096 0.15357072 0.11605511 0.1042691 ]
 [0.10064101 0.03572341 0.12739551 0.06259

Since the MIDI number labels of matrix are important to keep track of along with the matrix, a better saving and loading system from Numpy can be used to save and load multiple arrays:

In [17]:
#save the arrays and use labels/matrix as the keywords
np.savez("exampleChain.npz", labels = exampleMIDI, matrix = midiExampleMatrix)

#clear for testing
exampleMIDI = ["Hello World!"]
midiExampleMatrix = [["Hi!"]]
print("Cleared Version:")
print("labels: " + str(exampleMIDI))
print("matrix: " + str(midiExampleMatrix))

#load
exampleLoad = np.load("exampleChain.npz");
exampleMIDI = exampleLoad['labels']
midiExampleMatrix = exampleLoad['matrix']
print("Loaded Version:")
print("labels: " + str(exampleMIDI))
print("matrix: " + str(midiExampleMatrix))

Cleared Version:
labels: ['Hello World!']
matrix: [['Hi!']]
Loaded Version:
labels: [53 57 58 65 69 70 77 81 82 89 93 94]
matrix: [[0.16417593 0.11179552 0.13061582 0.02822897 0.00443455 0.13219276
  0.14926289 0.00407563 0.08121921 0.08691868 0.0984985  0.00858154]
 [0.14576113 0.11859473 0.13326446 0.08145893 0.13193554 0.01562802
  0.03565499 0.04213118 0.07622882 0.07480677 0.11554034 0.0289951 ]
 [0.08199023 0.02587868 0.11860992 0.1431949  0.09403055 0.02316978
  0.07553789 0.05821546 0.13091925 0.14056089 0.05921884 0.0486736 ]
 [0.08120194 0.02683207 0.01797798 0.09823133 0.11423344 0.0819366
  0.13903761 0.0905823  0.12692173 0.07077596 0.05023265 0.1020364 ]
 [0.07757943 0.0347367  0.03771359 0.12205095 0.07623134 0.14087784
  0.14553913 0.08209062 0.03887101 0.11097606 0.05656686 0.07676647]
 [0.03201435 0.04995772 0.09196333 0.04421644 0.10861992 0.11047114
  0.1204428  0.13421872 0.01516439 0.12868265 0.04748355 0.11676498]
 [0.16100608 0.00765774 0.03510095 0.05279642 0.1

The next function to build is one that will generate a list of midi pitches by calling the choose() method repeatedly:

In [18]:
#generates a list of MIDI pitches using a given Markov chain and list of values
#start is the index of the first midi pitch in the given list of MIDI values (default 0)
#length is how long the fragment should be (default 8)
def generateFragment(midi, matrix, start = 0, length = 8):
    curr = start
    fragment = [midi[start]]
    for i in range(length-1):
        row = matrix[curr]
        curr = choose(row)
        fragment.append(midi[curr])
        
    return fragment

In [19]:
#generate fragment from previous midi example
fragmentExample = generateFragment(exampleMIDI, midiExampleMatrix)
print(fragmentExample)

[53, 93, 65, 94, 93, 93, 69, 93]


Now this list can be used with Music21 to output a score and audio for it using a stream:

In [20]:
#generates a Music21 Stream from the list form of a musical fragment
def streamFromMIDI(fragment, title = "Fragment", name = "unknown"):
    result = stream.Stream();
    result.metadata = metadata.Metadata()
    result.metadata.composer = name
    result.metadata.title = title
    result.insert(0, instrument.Piano())
    result.append(note.Rest()) #this is necessary for proper playback in Jupyter
    for value in fragment:
        converted = note.Note(value)
        result.append(converted)
    
    return result

Here's a function which uses a showScore method obtained from this <a href='https://github.com/cuthbertLab/music21/issues/306'>GitHub</a> to show the score in Jupyter, as well as the default way to output midi in Music21

In [21]:
def show(fragment):
    showScore(fragment)
    fragment.show('midi')

In [22]:
#turn midi list into stream:
streamExample = streamFromMIDI(fragmentExample)
show(streamExample)

DIV_ID OSMD-div-908019


<IPython.core.display.Javascript object>

xml length: 3959


## User Interface:

Below is a user interface for easily changing parameters, along with the helper functions for making it work. I tried using widgets to make it more user-friendly, but something is wrong with my installation of it, and the widgets extension isn't working. Instead, there are fill in the variable boxes below (restart:

##### <center>MIDI Pitch Values with Keyboard</center>
![](notes.gif)

In [23]:
#edit these
seed = 99999 #change the random seed
pcs = {0,1,2,3,4,5,6,7,8,9,10,11} #remove pitch classes as wanted
minMIDI = 48 #currently set to C3
maxMIDI = 60 #currently set to C4
start = 0 #from set order: so if {01347} was given, 0=0 1=1 2=3 3=4 4=7
length = 8
title = "Hello World!"
composer = "Collin Presser"

In [24]:
#do not change
resMIDI = pcsToMIDI(pcs, minMIDI, maxMIDI)
print(resMIDI)
probMatrix = np.random.rand(len(resMIDI), len(resMIDI))
normalize(probMatrix)
printChain(resMIDI, probMatrix)

[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]


In [25]:
fragment = generateFragment(resMIDI, probMatrix, start, length)
stream = streamFromMIDI(fragment, title, composer)
show(stream)

DIV_ID OSMD-div-979951


xml length: 4091


<IPython.core.display.Javascript object>

## References:

<div style=“text-indent: -36px; padding-left: 36px;”>
    <p>Image of MIDI pitch values. <i>University of New South Wales.</i> n.d. Accessed May 6, 2021. https://newt.phys.unsw.edu.au/jw/notes.html</p>
    <p>Osipenko, Alexander. "Markov Chain for music generation." <i>towards data science.</i> July 27, 2019. Accessed May 6, 2021. https://towardsdatascience.com/markov-chain-for-music-generation-932ea8a88305</p>
</div>

## Further Reading:

<div style=“text-indent: -36px; padding-left: 36px;”>
    <p>Roads, C. "Artificial Intelligence and Music." <i>Computer Music Journal</i> 4, no. 2 (1980): 13-25. Accessed May 6, 2021. doi:10.2307/3680079.</p>
    <p>Serra, Xavier. "The Computational Study of a Musical Culture through Its Digital Traces." <i>Acta Musicologica</i> 89, no. 1 (2017): 24-44. Accessed May 6, 2021. https://www.jstor.org/stable/26350040.</p>
    <p>Wilf, Eitan. "Toward an Anthropology of Computer-Mediated, Algorithmic Forms of Sociality." <i>Current Anthropology</i> 54, no. 6 (2013): 716-39. Accessed May 6, 2021. doi:10.1086/673321.</p>
</div>